In [2]:
#Script reads in config file in directory to 
#Query data from rtap database
#Review Config File to access specfic database (current database set to 2017; port=5435)

import os
import psycopg2
from configparser import ConfigParser

import pandas as pd
import numpy as np

#bokeh
from bokeh.io import show, output_notebook, push_notebook, curdoc
from bokeh.plotting import figure

from bokeh.layouts import layout, column, row, WidgetBox
from bokeh.models import Panel, Spacer, HoverTool, LogColorMapper, ColumnDataSource, TapTool, BoxSelectTool, CustomJS
from bokeh.models.widgets import Div, Tabs, Paragraph, Dropdown, Button, PreText, Toggle,DataTable, DateFormatter, TableColumn
from bokeh.events import Tap

from bokeh.tile_providers import STAMEN_TERRAIN_RETINA,CARTODBPOSITRON_RETINA

#from lxml.html import parse

import bokeh

In [3]:
wdir = r'S:\AdminGroups\ResearchAnalysis\BER\ModelDataViz\TranDataArchive\data'

In [ ]:
vds_sensor = pd.read_csv(os.path.join(wdir,'location','vds_sensor_loc_2017.csv'))
il_vds = vds_sensor.loc[vds_sensor['STATE'] == 'IL']

display(il_vds.head(),len(il_vds))

In [4]:
#Function to collect and store data column names and types from rtdap database sources
def CollectDataFields(html="db_structure\db_struc.html"):
    DB_DataDict_list = []
    page = parse(html)
    
    #iterate over databases sources
    for db_sources in range(4,8):
        #identify tables for each db source
        tbls = len(page.xpath("/html/body/ul[{0}]".format(db_sources))[0].findall("li"))+1

        #iterate over tables in db sources and collect table column names and types
        for tbl in range(1,tbls):
            tbl_name = page.xpath("/html/body/ul[{0}]/li[{1}]".format(db_sources,tbl))[0].findall("h3")[0].text
            #print(tbl_name)
            rows = page.xpath("/html/body/ul[{0}]/li[{1}]/table".format(db_sources,tbl))[0].findall("tr")
            data = list()
            for row in rows:
                data.append([c.text for c in row.getchildren()])

            for row in data[2:]: 
                DB_DataDict_list.append([tbl_name] + row)

    return pd.DataFrame(DB_DataDict_list,columns=['TableName','ColumnName','DataType','Required','PrimaryKey'])

DB_DataDict = CollectDataFields()
DB_DataDict.to_csv('db_structure\db_fields.csv')
DB_DataDict.head()

NameError: name 'parse' is not defined

In [5]:
#read in config file containing connection parameters
def config(filename='rtdap_2017.ini',section='postgresql'):
    #create parser
    parser = ConfigParser()
    #read config file
    parser.read_file(open(filename))
    
    #get section, default to postgresql
    db={}
    if parser.has_section(section):
        params= parser.items(section)
        for param in params:
            #iterate over items in config file
            #to inject dbname, host, user, and password
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in {1} file'.format(section,filename))
        
    
    return db

#establish connection to database
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        #read in configuration parameters
        params = config()
        
        #connect to PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        
        #create cursor
        cur = conn.cursor()
        
    #execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        #display the PosgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

        #close the communication with the PostgreSQL 
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
 

 
if __name__ == '__main__':
    connect()
    

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


In [10]:
def select_table(data_table):
    """ query records from db table"""
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute('''
            select *
            from "''' + data_table + '''"
            limit 5''')
        result = [row for row in cur.fetchall()]
        names = [name[0] for name in cur.description]
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

    return pd.DataFrame(result,columns=names)

In [12]:
select_table('GatewayVDSDetail_2017')

,LocStatus,DetectorizationRatio,FieldDeviceID,IsSpeedTrap,Occupancy,DataStatus,OwningAgencyID,Type,Volume,LastUpdateTime,LocationTimeStamp,SourceID,DeviceStatus,ParentType,Speed
0,LOCATION_RESOLVED_AUTO,1.0,IL-IDOTD2-I-74 SB from 1st Ave to 4th Ave,False,7.142857,FIELD_DATA_VALIDATED_AUTO,IDOT D2,None,919.0,2017-04-25 13:01:00-05:00,2016-09-06 20:16:58-05:00,e69ac540b5cdd80aebd9c1767e8964b174d0fa0a,OPERATIONAL,"['VDS_DEVICETYPE', 'UNKNOWN_VDS_TYPE']",25.763890
1,LOCATION_RESOLVED_AUTO,1.0,IL-IDOTD2-I-74 SB from 4th Ave to 7th Ave_RAMP,False,2.071429,FIELD_DATA_VALIDATED_AUTO,IDOT D2,None,295.0,2017-04-25 13:01:00-05:00,2016-09-06 20:16:57-05:00,e69ac540b5cdd80aebd9c1767e8964b174d0fa0a,OPERATIONAL,"['VDS_DEVICETYPE', 'UNKNOWN_VDS_TYPE']",25.416668
2,LOCATION_RESOLVED_AUTO,1.0,IL-IDOTD2-I-74 NB from 1st Ave to Toll Pla,False,3.071429,FIELD_DATA_VALIDATED_AUTO,IDOT D2,None,340.0,2017-04-25 13:01:00-05:00,2016-09-06 20:16:57-05:00,e69ac540b5cdd80aebd9c1767e8964b174d0fa0a,OPERATIONAL,"['VDS_DEVICETYPE', 'UNKNOWN_VDS_TYPE']",17.222223
3,LOCATION_RESOLVED_AUTO,1.0,IL-IDOTD2-I-74 NB from 7th Ave to 4th Ave,False,2.133333,FIELD_DATA_INFEASIBLE_FOUND_AUTO,IDOT D2,None,366.0,2017-04-23 19:28:40-05:00,2017-04-05 12:19:39-05:00,e69ac540b5cdd80aebd9c1767e8964b174d0fa0a,COMMUNICATOINS_FAILURE,"['VDS_DEVICETYPE', 'UNKNOWN_VDS_TYPE']",29.611113
4,LOCATION_RESOLVED_AUTO,1.0,IL-IDOTD2-I-74 NB from South Tower to Nort,False,0.000000,FIELD_DATA_NOT_VALIDATED,IDOT D2,None,0.0,2017-04-25 13:01:26-05:00,2016-09-06 20:16:57-05:00,e69ac540b5cdd80aebd9c1767e8964b174d0fa0a,NON_OPERATIONAL,"['VDS_DEVICETYPE', 'UNKNOWN_VDS_TYPE']",0.000000


In [7]:
#return all results (limit first 10)
def select_table(data_table):
    """ query records from db table"""
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute('''
            select "FieldDeviceID",
            "Occupancy",
            "Volume",
            "Speed",
            extract(isodow from "LocationTimeStamp") AS "dow",
            extract(year from "LocationTimeStamp") AS "year",
            extract(month from "LocationTimeStamp") AS "month",
            extract(doy from "LocationTimeStamp") AS "doy",
            extract(hour from "LocationTimeStamp") AS "hour"
            from "''' + data_table + '''"
            where extract(isodow from "LocationTimeStamp") <  %s AND "DeviceStatus" = 'OPERATIONAL' 
            AND LEFT("OwningAgencyID",2) = 'IL' AND "FieldDeviceID" = 'IL-TESTTSC-STEVENSON-N-6027'
            limit 5'''% ("'6'"))
        result = [row for row in cur.fetchall()]
        names = [name[0] for name in cur.description]
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
    return pd.DataFrame(result,columns=names)

In [8]:
#return results
vds_detail = select_table("GatewayVDSDetail_2017")
vds_detail.head()

,FieldDeviceID,Occupancy,Volume,Speed,dow,year,month,doy,hour
0,IL-TESTTSC-STEVENSON-N-6027,4.632143,528.0,26.006962,1.0,2016.0,5.0,130.0,10.0
1,IL-TESTTSC-STEVENSON-N-6027,65.707146,324.0,1.397325,1.0,2016.0,5.0,130.0,10.0
2,IL-TESTTSC-STEVENSON-N-6027,65.257140,180.0,0.782251,1.0,2016.0,5.0,130.0,10.0
3,IL-TESTTSC-STEVENSON-N-6027,14.307143,1440.0,23.616318,1.0,2016.0,5.0,130.0,10.0
4,IL-TESTTSC-STEVENSON-N-6027,16.975000,1248.0,16.929893,1.0,2016.0,5.0,130.0,10.0


In [7]:
config_files = [('rtdap_2017.ini','GatewayVDSDetail_2017'),
                ('rtdap_2016.ini','GatewayVDSDetail_2016'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2015'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2014'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2013')]

def pull_summary(config_file,data_table):
    def config(filename=config_file,section='postgresql'):
        #create parser
        parser = ConfigParser()
        #read config file
        parser.read_file(open(filename))

        #get section, default to postgresql
        db={}
        if parser.has_section(section):
            params= parser.items(section)
            for param in params:
                #iterate over items in config file
                #to inject dbname, host, user, and password
                db[param[0]] = param[1]
        else:
            raise Exception('Section {0} not found in {1} file'.format(section,filename))


        return db

    #establish connection to database
    def connect():
        """ Connect to the PostgreSQL database server """
        conn = None
        try:
            #read in configuration parameters
            params = config()

            #connect to PostgreSQL server
            print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(**params)

            #create cursor
            cur = conn.cursor()

        #execute a statement
            print('PostgreSQL database version:')
            cur.execute('SELECT version()')

            #display the PosgreSQL database server version
            db_version = cur.fetchone()
            print(db_version)

            #close the communication with the PostgreSQL 
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()
                print('Database connection closed.')

    def select_table(data_table):
        """ query records from db table"""
        conn = None
        try:
            params = config()
            conn = psycopg2.connect(**params)
            cur = conn.cursor()
            cur.execute('''
                select "FieldDeviceID",
                "Occupancy",
                "Volume",
                "Speed",
                extract(isodow from "LocationTimeStamp") AS "dow",
                extract(year from "LocationTimeStamp") AS "year",
                extract(month from "LocationTimeStamp") AS "month",
                extract(doy from "LocationTimeStamp") AS "doy",
                extract(hour from "LocationTimeStamp") AS "hour"
                from "''' + data_table + '''"
                where extract(isodow from "LocationTimeStamp") <  %s AND "DeviceStatus" = 'OPERATIONAL' 
                AND LEFT("OwningAgencyID",2) = 'IL' AND "FieldDeviceID" = 'IL-TESTTSC-STEVENSON-N-6027'
                limit 5'''% ("'6'"))
            result = [row for row in cur.fetchall()]
            names = [name[0] for name in cur.description]
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()

        return pd.DataFrame(result,columns=names)


    #return all results (limit first 10)
    def select_vds(data_table):
        """ query records from db table"""
        conn = None
        try:
            params = config()
            conn = psycopg2.connect(**params)
            cur = conn.cursor()
            cur.execute('''
                select "FieldDeviceID",
                AVG("Occupancy"),
                AVG("Volume"),
                AVG("Speed"),
                extract(isodow from "LastUpdateTime") AS "dow",
                extract(year from "LastUpdateTime") AS "year",
                extract(month from "LastUpdateTime") AS "month",
                extract(doy from "LastUpdateTime") AS "doy",
                extract(hour from "LastUpdateTime") AS "hour"
                from "''' + data_table + '''"
                where extract(isodow from "LastUpdateTime") <  %s AND "DeviceStatus" = 'OPERATIONAL' 
                AND LEFT("FieldDeviceID",2) = 'IL'
                group by "FieldDeviceID", 
                extract(year from "LastUpdateTime"),
                extract(month from "LastUpdateTime"),
                extract(isodow from "LastUpdateTime"),
                extract(doy from "LastUpdateTime"),
                extract(hour from "LastUpdateTime")
                ''' % ("'6'"))
            result = [row for row in cur.fetchall()]
            names = [name[0] for name in cur.description]
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()

        return pd.DataFrame(result,columns=names)
    
    
    if __name__ == '__main__':
        connect()
    
    try:
        vds_table = select_vds(data_table)
        vds_table.to_csv(os.path.join(wdir,data_table+'.csv'),index=False)
    except:
        pass

    return display(vds_table.head())
    

In [8]:
for i in config_files:
    pull_summary(i[0],i[1])

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-EISENHOWER-E-3011,3.055357,333.000000,28.280670,4.0,2017.0,11.0,320.0,3.0
1,IL-TESTTSC-STEVENSON-N-6032,14.230953,1548.000000,24.698637,2.0,2017.0,1.0,24.0,17.0
2,IL-TESTTSC-BISHOP_FORD-S-7123,6.242262,649.000000,27.017583,3.0,2017.0,2.0,53.0,23.0
3,IL-TESTTSC-DAN_RYAN-S-5304,4.246230,508.333333,38.814847,5.0,2017.0,3.0,90.0,5.0
4,IL-TESTTSC-I_290_EXPRESS-W-4302,99.996728,0.000000,NaN,3.0,2017.0,3.0,81.0,10.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-IDOTD2-I-74 NB from 12th Ave to 7th Ave,0.000000,0.0,0.000000,2.0,2016.0,9.0,250.0,20.0
1,IL-IDOTD2-I-74 NB from 12th Ave to 7th Ave,0.262447,-1.0,28.607494,1.0,2016.0,10.0,277.0,0.0
2,IL-IDOTD2-I-74 NB from 12th Ave to 7th Ave,0.198810,-1.0,27.788747,1.0,2016.0,10.0,277.0,1.0
3,IL-IDOTD2-I-74 NB from 12th Ave to 7th Ave,0.220238,-1.0,26.918818,1.0,2016.0,10.0,277.0,2.0
4,IL-IDOTD2-I-74 NB from 12th Ave to 7th Ave,0.304729,-1.0,26.551202,1.0,2016.0,10.0,277.0,3.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-STEVENSON-S-6134,13.715972,335.083333,23.713448,5.0,2015.0,7.0,198.0,17.0
1,IL-TESTTSC-DAN_RYAN_EXPRESS-N-5020,1.570982,18.500000,17.699039,4.0,2015.0,1.0,22.0,6.0
2,IL-TESTTSC-STEVENSON-N-6024,14.612798,108.000000,22.477530,2.0,2015.0,5.0,146.0,18.0
3,IL-TESTTSC-STEVENSON-S-6122,9.430655,83.500000,24.286346,4.0,2015.0,4.0,99.0,19.0
4,IL-TESTTSC-I_55-N-6006,4.344345,45.333333,31.713139,2.0,2015.0,6.0,160.0,22.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-EDENS-S-1003,4.954707,144.750000,27.623694,3.0,2014.0,3.0,85.0,18.0
1,IL-TESTTSC-DAN_RYAN_EXPRESS-S-5187,4.490239,351.166667,35.558534,3.0,2014.0,7.0,183.0,22.0
2,IL-TESTTSC-LAKE_SHORE_DRIVE_NORTH-N-10304,6.875658,380.666667,24.390441,4.0,2014.0,6.0,177.0,20.0
3,IL-TESTTSC-STEVENSON-S-6122,1.396786,52.500000,28.158962,1.0,2013.0,12.0,364.0,3.0
4,IL-TESTTSC-LAKE_SHORE_DRIVE_SOUTH-S-10117,2.426408,36.812500,15.201591,5.0,2014.0,9.0,269.0,23.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-BISHOP_FORD-S-7123,5.330187,138.250000,20.702009,4.0,2013.0,11.0,325.0,5.0
1,IL-TESTTSC-I_290_EXPRESS-E-4202,6.051197,272.750000,30.680282,2.0,2013.0,9.0,246.0,9.0
2,IL-TESTTSC-KENNEDY-W-2108,2.028261,107.333333,27.659477,3.0,2013.0,3.0,72.0,2.0
3,IL-TESTTSC-ROUTE_53_EXPRESS-S-11202,6.866793,297.250000,28.664435,4.0,2013.0,4.0,94.0,15.0
4,IL-TESTTSC-I_55-S-6148,7.168749,237.000000,24.773160,5.0,2013.0,8.0,221.0,11.0


In [9]:
config_files = [('rtdap_2018.ini','GatewayVDSDetail_2018'),
                ('rtdap_2017.ini','GatewayVDSDetail_2017'),
                ('rtdap_2016.ini','GatewayVDSDetail_2016'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2015'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2014'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2013')]

vds_table = pd.DataFrame()

#format and join files
for i in config_files:
    df = pd.read_csv(os.path.join(wdir,i[1]+'.csv'))
    df.columns = ['FieldDeviceID','avgOccupancy','avgVolume','avgSpeed','dow','year','month','day','hour']
    vds_table = vds_table.append(df)
    

vds_table = vds_table.fillna(99999)    
vds_table.loc[(vds_table['avgSpeed']!=99999) &\
              (vds_table['avgVolume']!=99999) &\
              (vds_table['avgSpeed']!=99999)].groupby(['year']).agg({'FieldDeviceID':'count',
                                                                     'avgOccupancy':np.mean,
                                                                     'avgVolume':np.mean,
                                                                     'avgSpeed':np.mean})

,FieldDeviceID,avgOccupancy,avgVolume,avgSpeed
year,,,,
2011.0,4,0.000000,0.000000,22.798984
2012.0,809,1.825710,88.635352,31.175494
2013.0,4947516,7.747736,248.527087,25.467957
2014.0,5330821,7.544135,232.146991,26.492185
2015.0,4438869,10.477698,350.499357,25.829414
2016.0,4068541,10.902085,829.191571,26.125907
2017.0,3911676,10.462148,824.150997,25.721862
2018.0,65,6.434568,582.326637,24.406003


In [10]:
vds_table.to_csv(r'C:\Users\bross\Documents\GitHub\rtdap\rtdap_prototype\data\vds_table.csv')

In [4]:
vds_table_2017 = pd.read_csv(os.path.join(wdir,'GatewayVDSDetail_2017.csv'))

In [5]:
vds_loc = pd.read_csv(r'C:\Users\bross\Documents\GitHub\rtdap_bokeh_dashboard\rtdap_dashboard\data\sensor_2017_mhnc18q1_names.csv')

In [6]:
vds_loc.head()

,OBJECTID,NEAR_FID,NEAR_DIST,name_chk,dir_chk,Sensor_ID,MHN_ID,FID,OBJECTID_1,ANODE,BNODE,BASELINK,ABB,ROADNAME,POSTEDSPEE,MILES,Shape_Leng
0,1,349,11.088661,100,pass,IL-TESTTSC-STEVENSON-N-6027,11714-11843-1,166,20928,11714,11843,1,11714-11843-1,I-55-STEVENSON EXPY NB,55,0.792180,4182.709150
1,2,676,17.394368,100,pass,IL-TESTTSC-EDENS-N-1104,13702-20684-1,294,21852,13702,20684,1,13702-20684-1,I-94-EDENS EXPY WB,55,0.697714,3683.932312
2,3,660,0.283513,100,pass,IL-TESTTSC-EDENS-N-1126,13316-13201-1,279,21817,13316,13201,1,13316-13201-1,I-94-EDENS EXPY WB,55,1.325017,6996.087525
3,4,1104,3.475892,100,pass,IL-TESTTSC-DAN_RYAN_EXPRESS-N-5021,21088-15295-1,439,22937,21088,15295,1,21088-15295-1,I-90/94-DAN RYAN EXPY WB,45,0.562037,2967.557501
4,5,43,5.793484,100,pass,IL-TESTTSC-DAN_RYAN_EXPRESS-S-5102,21047-15275-1,20,17899,21047,15275,1,21047-15275-1,I-90/94-DAN RYAN EXPY EB,45,0.253048,1336.091900


In [7]:
vds_hwy_2017 = vds_table_2017.merge(vds_loc[['Sensor_ID','ABB','ROADNAME','MILES']], left_on ='FieldDeviceID', right_on = 'Sensor_ID')
vds_hwy_2017.head()

,FieldDeviceID,avg,avg.1,avg.2,dow,year,month,doy,hour,Sensor_ID,ABB,ROADNAME,MILES
0,IL-TESTTSC-EISENHOWER-E-3011,3.055357,333.0,28.280670,4.0,2017.0,11.0,320.0,3.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636
1,IL-TESTTSC-EISENHOWER-E-3011,24.376488,1569.0,15.223292,4.0,2017.0,11.0,313.0,15.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636
2,IL-TESTTSC-EISENHOWER-E-3011,15.646726,1587.0,23.389348,5.0,2016.0,12.0,344.0,13.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636
3,IL-TESTTSC-EISENHOWER-E-3011,16.371429,1643.0,24.281290,5.0,2017.0,8.0,216.0,5.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636
4,IL-TESTTSC-EISENHOWER-E-3011,29.777678,1394.0,11.353141,1.0,2017.0,6.0,170.0,13.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636


In [8]:
import datetime

vds_hwy_2017['date'] = pd.to_datetime(vds_hwy_2017.year.astype(int).astype(str)+'-1-1',format='%Y-%m-%d') + pd.to_timedelta(vds_hwy_2017.doy-1, 'd')
vds_hwy_2017.head()

,FieldDeviceID,avg,avg.1,avg.2,dow,year,month,doy,hour,Sensor_ID,ABB,ROADNAME,MILES,date
0,IL-TESTTSC-EISENHOWER-E-3011,3.055357,333.0,28.280670,4.0,2017.0,11.0,320.0,3.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636,2017-11-16
1,IL-TESTTSC-EISENHOWER-E-3011,24.376488,1569.0,15.223292,4.0,2017.0,11.0,313.0,15.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636,2017-11-09
2,IL-TESTTSC-EISENHOWER-E-3011,15.646726,1587.0,23.389348,5.0,2016.0,12.0,344.0,13.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636,2016-12-09
3,IL-TESTTSC-EISENHOWER-E-3011,16.371429,1643.0,24.281290,5.0,2017.0,8.0,216.0,5.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636,2017-08-04
4,IL-TESTTSC-EISENHOWER-E-3011,29.777678,1394.0,11.353141,1.0,2017.0,6.0,170.0,13.0,IL-TESTTSC-EISENHOWER-E-3011,12978-21065-1,I-290-EISENHOWER EXPY EB,0.4636,2017-06-19


In [13]:
#groupby and get average speed and occupancy and sum of volume per mhn link
vds_hwy_2017['missing_occ'] = np.where(vds_hwy_2017['avg'].isnull(),1,0)
vds_hwy_2017['missing_vol'] = np.where(vds_hwy_2017['avg.1'].isnull(),1,0)
vds_hwy_2017['missing_speed'] = np.where(vds_hwy_2017['avg.2'].isnull(),1,0)
vds_hwy_2017['frequency'] = 1


ABB_vds = vds_hwy_2017.groupby(['ABB','ROADNAME','MILES','dow','year','month','doy','hour']).agg({'avg':np.mean, 
                                                                                                  'avg.1':sum,
                                                                                                  'avg.2':np.mean,
                                                                                                 'missing_occ':sum,
                                                                                                 'missing_vol':sum,
                                                                                                 'missing_speed':sum,
                                                                                                 'frequency':sum}).reset_index()

#calculate vmt per link using vds volume (summed to mhn link level) and mhn link length
ABB_vds['avg_vmt'] = ABB_vds['MILES'] * ABB_vds['avg.1']

#groupby road name and calculate average speed and occupancy along the corridor and sum volume, vmt, and length
hwy_vds = ABB_vds.groupby(['ROADNAME','dow','year','month','doy','hour']).agg({'avg':np.mean, 
                                                                               'avg.1':sum,
                                                                               'avg.2':np.mean,
                                                                               'avg_vmt':sum,
                                                                               'MILES':sum,
                                                                               'missing_occ':sum,
                                                                               'missing_vol':sum,
                                                                               'missing_speed':sum,
                                                                              'frequency':sum}).reset_index()

In [14]:
#every day of the year not avaiable for each hwy
display('total vds sensors ' + str(len(vds_hwy_2017)) + ' ' + 'total mhn links ' + str(len(ABB_vds)) +\
        ' ' + 'total corriodr records ' + str(len(hwy_vds)))

'total vds sensors 4872398 total mhn links 3152820 total corriodr records 270025'

In [20]:
ABB_vds.columns

Index(['ABB', 'ROADNAME', 'MILES', 'dow', 'year', 'month', 'doy', 'hour',
       'avg', 'avg.1', 'avg.2', 'missing_occ', 'missing_vol', 'missing_speed',
       'frequency', 'avg_vmt', 'date'],
      dtype='object')

In [21]:
ABB_vds['date'] = pd.to_datetime(ABB_vds.year.astype(int).astype(str)+'-1-1',format='%Y-%m-%d') + pd.to_timedelta(ABB_vds.doy-1, 'd')
ABB_vds['ROADNAME'] = ABB_vds['ROADNAME'].str.replace(" / ","-")

ABB_vds.columns = ['ABB', 'ROADNAME', 'MILES', 'dow', 'year', 'month', 'doy', 
         'hour','avg_occupancy','avg_volume','avg_speed', 'missing_occ', 
         'missing_vol', 'missing_speed','frequency', 'avg_vmt', 'date']

ABB_vds.to_csv(r'C:\Users\bross\Documents\GitHub\rtdap_bokeh_dashboard\rtdap_dashboard\data\test.csv.gz',index=False,compression='gzip')

In [120]:
hwy_vds['date'] = pd.to_datetime(hwy_vds.year.astype(int).astype(str)+'-1-1',format='%Y-%m-%d') + pd.to_timedelta(hwy_vds.doy-1, 'd')
hwy_vds['ROADNAME'] = hwy_vds['ROADNAME'].str.replace(" / ","-")

hwy_vds.to_csv(r'C:\Users\bross\Documents\GitHub\rtdap_bokeh_dashboard\rtdap_dashboard\data\test.csv.gz',index=False,compression='gzip')

In [115]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
pd.read_csv(r'test.csv.gz', compression='gzip', dtype={'hour':int}, parse_dates=['date'], date_parser=dateparse)

,ROADNAME,dow,date,hour,frequency,avg_occupancy,avg_volume,avg_speed,avg_vmt,MILES,missing_occ,missing_vol,missing_speed
0,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,0,6,7.022321,5046.666667,27.660404,1050.709110,1.082428,0,0,0
1,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,1,6,3.308413,2443.500000,29.287751,508.712447,1.082428,0,0,0
2,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,2,6,1.503244,1142.000000,30.295753,239.801078,1.082428,0,0,0
3,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,3,6,1.303403,947.666667,29.884419,199.747089,1.082428,0,0,0
4,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,4,6,3.154266,2473.333333,30.198982,518.900916,1.082428,0,0,0
5,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,5,6,7.478423,5647.333333,28.358854,1184.076312,1.082428,0,0,0
6,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,6,6,9.417917,6194.166667,25.737899,1294.064020,1.082428,0,0,0
7,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,7,6,7.289702,5155.666667,27.436725,1076.326436,1.082428,0,0,0
8,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,8,6,7.708581,5177.166667,26.763728,1079.840665,1.082428,0,0,0
9,I-190 / KENNEDY EXPY EB,1.0,2016-12-05,9,6,8.222242,5428.500000,26.414898,1130.759263,1.082428,0,0,0
